In [19]:
import requests
import json
url = "http://localhost:8000/whisperX/"
# open the JSON file to upload
#aligned json is optional and must only be passed if u already have the json file

with open("../notebook/fold.json", "r") as input_file:
    data = json.load(input_file)
files = {'audio': ('../audio/fold.wav', open('../audio/fold.wav', 'rb'), 'audio/wav'),
         'aligned_json': ("../notebook/fold.json", json.dumps(data), "application/json")}
data = {'transcription': 'srt', 'model_name': 'tiny','translate':False,
        "device":"cpu",
        "language":"en","patience":1.0,"temperature":1.0,
        "condition_on_previous/_text":False,"temperature_increment_on_fallback":0.2,
        "compression_ratio_threshold":2.4,"logprob_threshold":-1.0,"no_speech_threshold":0.6
        }

response = requests.post(url, files=files, data=data)

response.json()

{'segments_before_alignment': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 4.34,
   'text': " Today I'm going to show you how to fold a shirt.",
   'tokens': [2692,
    286,
    478,
    516,
    281,
    855,
    291,
    577,
    281,
    4860,
    257,
    8336,
    13],
   'temperature': 0.7,
   'avg_logprob': -0.4336632995605469,
   'compression_ratio': 1.800751879699248,
   'no_speech_prob': 0.03584390878677368,
   'seg-text': [" Today I'm going to show you how to fold a shirt."]},
  {'id': 1,
   'seek': 0,
   'start': 4.34,
   'end': 11.540000000000001,
   'text': ' in under two seconds. Start by taking a short sleeve shirt and lying it out on us back.',
   'tokens': [294,
    833,
    732,
    3949,
    13,
    6481,
    538,
    1940,
    257,
    2099,
    21138,
    8336,
    293,
    8493,
    309,
    484,
    322,
    505,
    646,
    13],
   'temperature': 0.7,
   'avg_logprob': -0.4336632995605469,
   'compression_ratio': 1.800751879699248,
   'no_speech_prob': 